In [20]:
from datasets import load_dataset

stack_exchange_data = load_dataset("HuggingFaceH4/stack-exchange-preferences",split="train",streaming=True)
stackexchange_subset_data = stack_exchange_data.take(1000_000)

In [21]:
stack_exchange_data_list = list(stackexchange_subset_data)

In [30]:
sd = stack_exchange_data_list[10050]

In [31]:
sd

{'qid': 74880,
 'question': '<p>Conceptually, I would like to accomplish the following but have had trouble understand how to code it properly in C#:</p>\n\n<pre><code>\nSomeMethod { // Member of AClass{}\n    DoSomething;\n    Start WorkerMethod() from BClass in another thread;\n    DoSomethingElse;\n}\n</code></pre>\n\n<p>Then, when WorkerMethod() is complete, run this:</p>\n\n<p><pre><code>\nvoid SomeOtherMethod()  // Also member of AClass{}\n{ ... }\n</pre></code></p>\n\n<p>Can anyone please give an example of that? </p>\n',
 'answers': [{'answer_id': 74917,
   'author': 'MagicKat',
   'author_id': 8505,
   'author_profile': 'https://Stackoverflow.com/users/8505',
   'pm_score': 1,
   'selected': False,
   'text': '<p>Check out BackgroundWorker.</p>\n'},
  {'answer_id': 74948,
   'author': 'Isak Savo',
   'author_id': 8521,
   'author_profile': 'https://Stackoverflow.com/users/8521',
   'pm_score': 5,
   'selected': True,
   'text': '<p>The <a href="http://msdn.microsoft.com/en-us/

In [24]:
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
def get_text_html(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    all_text = soup.get_text()
    return all_text

columns = ['date','qid','question','answer_text','selected_answer_text','stack_exchange']
df = pd.DataFrame(columns=columns)

qids = []
dates = []
question_list = []
answer_text_list = []
selected_answer_text_list = []
stack_exchange_list = []
for sd in tqdm(stack_exchange_data_list):
    best_answer_pm_score_list = [s['pm_score'] for s in sd['answers']]
    if len(list(set(best_answer_pm_score_list))) < len(best_answer_pm_score_list): continue
    if "meta" in sd['metadata'][0]:continue

    qids.append(sd['qid'])
    dates.append(sd['date'])
    link = sd['metadata'][1]
    stack_exchange_list.append(link[link.find("//"):link.find(".")][2:])
    question_list.append(get_text_html(sd['question']))
    answer_text = ""
    best_answer = ""
    best_answer_pm_score = 0
    for idx,ans in enumerate(sd['answers']):
        answer_text += get_text_html(ans['text']) + "\n\n\n\n"
        if ans['pm_score'] > best_answer_pm_score:
            best_answer = answer_text.split("\n\n\n\n")[0]

    answer_text_list.append(answer_text)
    selected_answer_text_list.append(best_answer)

In [25]:

df['qid'] = qids
df['date'] = dates
df['question'] = question_list
df['answer_text'] = answer_text_list
df['selected_answer_text'] = selected_answer_text_list
df['stack_exchange'] = stack_exchange_list
df.head()

,date,qid,question,answer_text,selected_answer_text,stack_exchange
0,2016/01/12,1,When I've printed an object I've had to choose...,You could experiment with slicing. For example...,You could experiment with slicing. For example...,3dprinting
1,2016/01/12,3,I know the minimum layer height will effect ho...,"As with any manufacturing process, you'll need...","As with any manufacturing process, you'll need...",3dprinting
2,2016/01/12,19,"My printed parts consist rafts, supports and o...",The best way to get rid of them is to change t...,The best way to get rid of them is to change t...,3dprinting
3,2016/01/12,20,I would like to understand the differences bet...,All three of these features are used to improv...,All three of these features are used to improv...,3dprinting
4,2016/01/12,22,I've acquired all the parts to build a Reprap ...,"Generally speaking, MDF will weather OK. In ar...","Generally speaking, MDF will weather OK. In ar...",3dprinting


In [26]:
df['stack_exchange'].value_counts()

stack_exchange
Stackoverflow    489681
3dprinting         1283
Name: count, dtype: int64

In [43]:
import csv
df.to_csv("StackExchangeData.csv",index=False,quoting=csv.QUOTE_NONE, escapechar='\\')

: 

In [37]:
print(len(df['answer_text'].values[0].split("\n\n\n\n")))

3


In [42]:
df['answer_text'].values[0].split("\n\n\n\n")[-1]

'\n'